# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gowurdak,37.8124,66.0466,13.08,60,16,0.25,TM,1700358749
1,1,cumaribo,4.4455,-69.7990,26.22,82,65,1.22,CO,1700358750
2,2,mishan,45.5500,131.8833,-3.32,79,99,3.35,CN,1700358751
3,3,red bank,33.9321,-81.2384,18.79,68,75,0.00,US,1700358752
4,4,ust-nera,64.5697,143.2370,-23.78,98,97,0.12,RU,1700358753


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
world_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    xlim = (-180,180),
    ylim = (-75,90),
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
world_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filtered_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_filtered_df = city_data_filtered_df.dropna()

# Display sample data
city_data_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,mangrol,21.1227,70.1148,22.73,51,0,3.98,IN,1700358794
54,54,tamandare,-8.7563,-35.1000,25.18,82,0,3.93,BR,1700358806
182,182,tranovaho,-25.3000,44.9667,22.20,74,0,1.22,MG,1700358928
192,192,skeldon,5.8833,-57.1333,25.46,84,0,2.79,GY,1700358939
318,318,sao jose da coroa grande,-8.8978,-35.1478,25.13,83,0,3.83,BR,1700359038
389,389,hayma',19.9593,56.2758,22.43,76,0,2.19,OM,1700359090
425,425,cidade velha,14.9153,-23.6053,26.21,22,0,3.60,CV,1700359114
505,505,tazacorte,28.6419,-17.9339,23.82,30,0,4.02,ES,1700359170
524,524,ondjiva,-17.0667,15.7333,23.02,64,0,3.09,AO,1700359184
535,535,arauca,7.0847,-70.7591,26.98,94,0,0.00,CO,1700359191


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
42,mangrol,IN,21.1227,70.1148,51,
54,tamandare,BR,-8.7563,-35.1000,82,
182,tranovaho,MG,-25.3000,44.9667,74,
192,skeldon,GY,5.8833,-57.1333,84,
318,sao jose da coroa grande,BR,-8.8978,-35.1478,83,
389,hayma',OM,19.9593,56.2758,76,
425,cidade velha,CV,14.9153,-23.6053,22,
505,tazacorte,ES,28.6419,-17.9339,30,
524,ondjiva,AO,-17.0667,15.7333,64,
535,arauca,CO,7.0847,-70.7591,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    location_url = base_url + f"?categories={params['categories']}&filter={params['filter']}&bias={params['bias']}&limit={params['limit']}&apiKey={params['apiKey']}"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(location_url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mangrol - nearest hotel: No hotel found
tamandare - nearest hotel: Pousada Baia dos Corais
tranovaho - nearest hotel: No hotel found
skeldon - nearest hotel: Classic International Hotel
sao jose da coroa grande - nearest hotel: Hotel do Sol
hayma' - nearest hotel: Arabian Sands
cidade velha - nearest hotel: Baía do Coral
tazacorte - nearest hotel: Apartamentos Atlantis
ondjiva - nearest hotel: Águia Verde Hotel
arauca - nearest hotel: Hotel Nova Park
nuriootpa - nearest hotel: The Vine Inn
fort pierce - nearest hotel: Dockside Inn
bandar-e lengeh - nearest hotel: Old owner do not like foreigners hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
42,mangrol,IN,21.1227,70.1148,51,No hotel found
54,tamandare,BR,-8.7563,-35.1000,82,Pousada Baia dos Corais
182,tranovaho,MG,-25.3000,44.9667,74,No hotel found
192,skeldon,GY,5.8833,-57.1333,84,Classic International Hotel
318,sao jose da coroa grande,BR,-8.8978,-35.1478,83,Hotel do Sol
389,hayma',OM,19.9593,56.2758,76,Arabian Sands
425,cidade velha,CV,14.9153,-23.6053,22,Baía do Coral
505,tazacorte,ES,28.6419,-17.9339,30,Apartamentos Atlantis
524,ondjiva,AO,-17.0667,15.7333,64,Águia Verde Hotel
535,arauca,CO,7.0847,-70.7591,94,Hotel Nova Park


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
ideal_locations_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    xlim = (-180,180),
    ylim = (-75,90),
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = "all"
)

# Display the map
ideal_locations_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,index,Country,Humidity,Hotel Name)